In [1]:
import pandas as pd
import numpy as np
import re
import string

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

from flatlib.datetime import Datetime
from flatlib.geopos import GeoPos
from flatlib.chart import Chart
from flatlib import const

from timezonefinder import TimezoneFinder
from datetime import datetime
from pytz import timezone, utc


In [2]:
df = pd.read_csv('some-csvs-to-work-with/hi_its_fashion-models_df.csv')

In [3]:
df.head()

,name,info,occupation
0,Carrie-Anne_Moss,Gender:FemaleDate of Birth:21 August 1967 - 1...,r_fashion-models
1,Cameron_Diaz,Gender:FemaleDate of Birth:30 August 1972 - 0...,r_fashion-models
2,Milla_Jovovich,Gender:FemaleDate of Birth:17 December 1975 -...,r_fashion-models
3,Natalie_Portman,Gender:FemaleDate of Birth:9 June 1981 - 15:4...,r_fashion-models
4,Kim_Basinger,Gender:FemaleDate of Birth:8 December 1953 - ...,r_fashion-models


In [4]:
copy_df = df
copy_df['count'] = np.arange(len(copy_df))

In [5]:
def adios_weird_characters(i):
    s = copy_df.iloc[i].info
    printable = set(string.printable)
    return ''.join(filter(lambda x: x in printable, s))

In [6]:
copy_df['info_norm'] = copy_df['count'].apply(adios_weird_characters)
#copy_df_mod = copy_df.drop(['info'], axis=1)
copy_df_mod = copy_df
copy_df_mod = copy_df_mod.drop(['info'], axis=1)
copy_df_mod = copy_df_mod.rename(columns={"info_norm": "info"})

In [7]:
copy_df_mod.head()

,name,occupation,count,info
0,Carrie-Anne_Moss,r_fashion-models,0,Gender:FemaleDate of Birth:21 August 1967 - 1...
1,Cameron_Diaz,r_fashion-models,1,Gender:FemaleDate of Birth:30 August 1972 - 0...
2,Milla_Jovovich,r_fashion-models,2,Gender:FemaleDate of Birth:17 December 1975 -...
3,Natalie_Portman,r_fashion-models,3,Gender:FemaleDate of Birth:9 June 1981 - 15:4...
4,Kim_Basinger,r_fashion-models,4,Gender:FemaleDate of Birth:8 December 1953 - ...


In [23]:
def get_birthtime_info(row):
    b_info = copy_df_mod['info'][row]
    try:
        if '*' in b_info:
            return None
        if b_info == None:
            return None
        if '(gregor.)' in b_info:
            #b_info_mod_time = [b_info_mod_time[0][:-9]]
            b_info = b_info.replace('(gregor.)','')
        elif 'h(gregor.' in b_info:
            #b_info_mod_time = b_info_mod_time[:-8]
            b_info = b_info.replace('(gregor.','')
        b_info_mod = ''
        if 'Gender:FemaleDate of Birth:' in b_info:
            b_info_mod = b_info.replace('Gender:FemaleDate of Birth:', '')
        elif 'Gender:MaleDate of Birth:' in b_info:
            b_info_mod = b_info.replace('Gender:MaleDate of Birth:', '')
        if 'Birth place:' in b_info_mod:
            b_info_mod = b_info_mod.split('Birth place:')
        elif 'Country:' in b_info_mod:
            b_info_mod = b_info_mod.split('Country:')
        b_info_mod_time = b_info_mod[0]

        b_info_mod_time = b_info_mod_time.split(' - ')
        #print(b_info_mod_time)
        #print('...end of get_birthtime_info')

        return b_info_mod_time
    except:
        return None

def get_birthplace_info(row):
    b_info = copy_df_mod['info'][row]
    try:
        if '*' in b_info:
            return None
        if b_info == None:
            return None
        if '(gregor.)' in b_info:
            #b_info_mod_place = b_info_mod_place[:-9]
            b_info = b_info.replace('(gregor.)','')
        elif '(gregor.' in b_info:
            #b_info_mod_place = b_info_mod_place[:-8]
            b_info = b_info.replace('h(gregor.','')
        b_info_mod = ''
        if 'Gender:FemaleDate of Birth:' in b_info:
            b_info_mod = b_info.replace('Gender:FemaleDate of Birth:', '')
        elif 'Gender:MaleDate of Birth:' in b_info:
            b_info_mod = b_info.replace('Gender:MaleDate of Birth:', '')
        else:
            return None
        if 'Birth place:' in b_info_mod:
            b_info_mod = b_info_mod.split('Birth place:')
        elif 'Country:' in b_info_mod:
            b_info_mod = b_info_mod.split('Country:')
        b_info_mod_place = b_info_mod[1].split('Age:')
        b_info_mod_place = b_info_mod_place[0]
        b_info_mod_place = b_info_mod_place.split('Country:')
        #print(b_info_mod_place)
        #print('...end of get_birthplace_info')

        return b_info_mod_place
    except:
        return None

def convert_month_name_to_number(month):
    month = month.lower()
    if month == 'january':
        return 1
    elif month == 'february':
        return 2
    elif month == 'march':
        return 3
    elif month == 'april':
        return 4
    elif month == 'may':
        return 5
    elif month == 'june':
        return 6
    elif month == 'july':
        return 7
    elif month == 'august':
        return 8
    elif month == 'september':
        return 9
    elif month == 'october':
        return 10
    elif month == 'november':
        return 11
    elif month == 'december':
        return 12
    else:
        print('Not_a_month')
        return None

def time_formatter(time_lst):
    if time_lst == None:
        return None
    if len(time_lst) == 2:
        first = time_lst[0]
        month = " ".join(re.split("[^a-zA-Z]*", first))
        month = month.replace(' ', '')
        first = first.replace(' ', '')
        temp = first.split(month)
        day = temp[0]
        year = temp[1]
        time = time_lst[1]
        time = time[:-1]
        ans = [year, convert_month_name_to_number(month), day, time]
        print(ans)
        return ans
    elif len(time_lst) == 1:
        first = time_lst[0]
        month = " ".join(re.split("[^a-zA-Z]*", first))
        month = month.replace(' ', '')
        #print(month)
        first = first.replace(' ', '')
        temp = first.split(month)
        day = temp[0]
        year = temp[1]
        ans = [year, convert_month_name_to_number(month), day]
        print(ans)
        return ans
    print('No_time_info')
    return None
        
    
def place_formatter(place_lst):
    if place_lst == None:
        return None
    if len(place_lst) == 1:
        name = place_lst[0].split(', ')
        ans = name[1]
        return ans
    else:
        city_info = place_lst[0]
        country_info = place_lst[1]
        if ',' in city_info:
            city_info = city_info.split(', ')
        else:
            city_info = [city_info]
        if ',' in country_info:
            country_info = country_info.split(', ')
            country_info = country_info[1]
        if len(city_info) == 2:
            city = city_info[0]
            state = city_info[1]
        #elif len(city_info) == 1:
        else:
            city = city_info[0]
            state = 'no_state'
        country = country_info
        if state == 'no_state':
            ans = [city, country]
            ans = ', '.join(ans)
        else:
            ans = [city, state, country]
            ans = ', '.join(ans)
        print(ans)
        return ans

def make_time_column(row):
    return time_formatter(get_birthtime_info(row))

def make_place_column(row):
    return place_formatter(get_birthplace_info(row))

In [24]:
print(get_birthtime_info(0))
#print(make_time_column(0))
#convert_month_name_to_number('May')

['21 August  1967', '16:05h']


In [25]:
#copy_df_mod['birthtime'] = copy_df_mod['count'].apply(make_time_column)
'''
for _ in range(80):
    print(copy_df_mod.iloc[_].info)
    print(get_birthtime_info(_))
    print(make_time_column(_))


print(copy_df_mod.iloc[76].info)
print(get_birthtime_info(76))
print(copy_df_mod.iloc[77].info)
print(get_birthtime_info(77))
print(copy_df_mod.iloc[78].info)
print(get_birthtime_info(78))
print(copy_df_mod.iloc[79].info)
print(get_birthtime_info(79))
print(copy_df_mod.iloc[80].info)
print(get_birthtime_info(80))
print(copy_df_mod.iloc[81].info)
print(get_birthtime_info(81))
print(copy_df_mod.iloc[82].info)
print(get_birthtime_info(82))
'''
copy_df_mod['birthplace'] = copy_df_mod['count'].apply(make_place_column)


'''
for _ in range(80):
    print(copy_df_mod.iloc[_].info)
    print(get_birthplace_info(_))
    print(make_place_column(_))


print(copy_df_mod.iloc[32].info)
print(get_birthplace_info(32))
print(place_formatter(get_birthplace_info(32)))
print(copy_df_mod.iloc[33].info)
print(get_birthplace_info(33))
print(place_formatter(get_birthplace_info(33)))
'''

Vancouver, British Columbia, Canada
San Diego, California, United States
Kiev, Ukraine
Jerusalem, Israel
Athens, Georgia, United States
Burbank, California, United States
Benoni, South Africa
Cleveland, Ohio, United States
Birmingham, Alabama, United States
Hampshire, England, United Kingdom
New Orleans, Louisiana, United States
Paris, France
Los Angeles, California, United States
Saint-Maur-des-Fosss, France
Chicago Heights, Illinois, United States
Ladysmith, British Columbia, Canada
New Ulm, Minnesota, United States
New York, New York, United States
Rome, Italy
Cincinnati, Ohio, United States
Paris, France
Ely, Minnesota, United States
Copenhagen, Denmark
Pomona, California, United States
New York, New York, United States
DeKalb, Illinois, United States
Houston, Texas, United States
Houston, Texas, United States
Inglewood, California, United States
Los Angeles, California, United States
Los Angeles, California, United States
New York, New York, United States
Houston, Texas, United St

San Francisco, California, United States
Miami, Florida, United States
Jackson, Iowa, United States
Los Angeles, California, United States
Bahia, Brazil
New York, New York, United States
Chtillon-sur-Seine, France
Moscow, Russia
Inglewood, California, United States
Hanover, New Hampshire, United States
Santa Rosa, Brazil
Long Branch, New Jersey, United States
Porto, Portugal
Ruskington, England, United Kingdom
So Paulo, Brazil
Tokyo, Japan
Phoenix, Arizona, United States
L'Aquila, Italy
Malm, Sweden
Porto, Portugal
Coral Gables, Florida, United States
Berlin, Germany
Phoenix, Arizona, United States
Newport Beach, California, United States
Philadelphia, Pennsylvania, United States
Bilbao, Spain
Marikina, Philippines
Santa Maria, Philippines
Nicosia, Cyprus
Budapest, Hungary
Washington, District of Columbia, United States
Yarrawonga, Victoria, Australia
Manila, Philippines
Almelo, Netherlands
Silsbee, Texas, United States
Gainesville, Florida, United States
Taipei, Taiwan
Davao City, Phi

'\nfor _ in range(80):\n    print(copy_df_mod.iloc[_].info)\n    print(get_birthplace_info(_))\n    print(make_place_column(_))\n\n\nprint(copy_df_mod.iloc[32].info)\nprint(get_birthplace_info(32))\nprint(place_formatter(get_birthplace_info(32)))\nprint(copy_df_mod.iloc[33].info)\nprint(get_birthplace_info(33))\nprint(place_formatter(get_birthplace_info(33)))\n'

In [140]:
copy_df_mod.head(100)

,name,occupation,count,info,birthtime,birthplace
0,Michael_Haneke,r_philosophers,0,Gender:MaleDate of Birth:23 March 1942Birth p...,"[1942, 1, 23]","Munich, Germany"
1,Noam_Chomsky,r_philosophers,1,Gender:MaleDate of Birth:7 December 1928 - 07...,"[1928, 1, 7, 07:00]","Philadelphia, Pennsylvania, United States"
2,Albert_Camus,r_philosophers,2,Gender:MaleDate of Birth:7 November 1913 - 02...,"[1913, 1, 7, 02:00]","Mondovi, Algeria"
3,Mahatma_Gandhi,r_philosophers,3,Gender:MaleDate of Birth:2 October 1869 - 07:...,"[1869, 1, 2, 07:08]","Porbandar, India"
4,Jean-Jacques_Rousseau,r_philosophers,4,Gender:MaleDate of Birth:28 June 1712 - 10:00...,"[1712, 1, 28, 10:00]","Geneva, Switzerland"
5,Denis_Diderot,r_philosophers,5,Gender:MaleDate of Birth:5 October 1713 - 17:...,"[1713, 1, 5, 17:00]","Langres, France"
6,Immanuel_Kant,r_philosophers,6,Gender:MaleDate of Birth:22 April 1724 - 05:0...,"[1724, 1, 22, 05:00]","Knigsberg, Germany"
7,Johann_Gottfried_Herder,r_philosophers,7,Gender:MaleDate of Birth:25 August 1744 - 23:...,"[1744, 1, 25, 23:30]","Mohrungen, Poland"
8,Soeren_Kierkegaard,r_philosophers,8,Gender:MaleDate of Birth:5 May 1813Birth plac...,"[1813, 1, 5]","Copenhagen, Denmark"
9,Friedrich_Nietzsche,r_philosophers,9,Gender:MaleDate of Birth:15 October 1844 - 10...,"[1844, 1, 15, 10:00]","Roecken, Germany"


In [12]:
strngstrng = '''
['1975', 1, '27', '16:35']
['1966', 1, '8', '18:05']
['1912', 1, '15', '23:30']
['1968', 1, '25']
['1904', 1, '7', '01:21']
['1962', 1, '26', '00:25']
['1956', 1, '13', '15:00']
['1846', 1, '28', '10:00']
['1959', 1, '15', '07:40']
['1964', 1, '13', '05:30']
['1964', 1, '13', '06:00']
['1956', 1, '25']
['1964', 1, '10']
['1935', 1, '18']
['1906', 1, '17']
['1949', 1, '8']
['1960', 1, '22']
['1975', 1, '29']
['1785', 1, '31']
['1967', 1, '7']
['1952', 1, '3']
['1975', 1, '9']
['1947', 1, '21']
['1960', 1, '19']
['1903', 1, '5']
['1968', 1, '14']
['1962', 1, '30']
['1969', 1, '21']
['1984', 1, '7']
['1977', 1, '5']
['1923', 1, '6']
['1971', 1, '19']
['1967', 1, '3']
['1968', 1, '1']
['1970', 1, '22']
['1947', 1, '19']
['1966', 1, '19']
['1975', 1, '13']
['1960', 1, '29']
['1943', 1, '5']
['1966', 1, '12']
['1968', 1, '22']
['1971', 1, '3']
['1931', 1, '29']
['1956', 1, '21']
['1948', 1, '2']
['1974', 1, '17']
['1975', 1, '5']
['1909', 1, '23']
['1965', 1, '10']
['1965', 1, '14']
['1966', 1, '3']
['1978', 1, '18']
['1955', 1, '2']
['1966', 1, '23']
['1988', 1, '1']
['1962', 1, '30']
['1952', 1, '19']
['1964', 1, '20']
['1973', 1, '6']
['1904', 1, '17']
['1945', 1, '23']
['1980', 1, '8']
['1940', 1, '20']
['1921', 1, '10']
['1953', 1, '11']
['1970', 1, '25']
['1976', 1, '12']
['1971', 1, '26']
['1977', 1, '26']
['1949', 1, '24']
['1939', 1, '22']
['1984', 1, '19']
['1958', 1, '1']
['1918', 1, '31']
['1969', 1, '19']
['1988', 1, '26']
'''
counttemptemp = 0
for e in strngstrng:
    if e == '\n':
        counttemptemp += 1
print(counttemptemp)

78


In [141]:
best_df_ever = copy_df_mod.drop(['info'], axis=1)

In [142]:
best_df_ever.head(15)

,name,occupation,count,birthtime,birthplace
0,Michael_Haneke,r_philosophers,0,"[1942, 1, 23]","Munich, Germany"
1,Noam_Chomsky,r_philosophers,1,"[1928, 1, 7, 07:00]","Philadelphia, Pennsylvania, United States"
2,Albert_Camus,r_philosophers,2,"[1913, 1, 7, 02:00]","Mondovi, Algeria"
3,Mahatma_Gandhi,r_philosophers,3,"[1869, 1, 2, 07:08]","Porbandar, India"
4,Jean-Jacques_Rousseau,r_philosophers,4,"[1712, 1, 28, 10:00]","Geneva, Switzerland"
5,Denis_Diderot,r_philosophers,5,"[1713, 1, 5, 17:00]","Langres, France"
6,Immanuel_Kant,r_philosophers,6,"[1724, 1, 22, 05:00]","Knigsberg, Germany"
7,Johann_Gottfried_Herder,r_philosophers,7,"[1744, 1, 25, 23:30]","Mohrungen, Poland"
8,Soeren_Kierkegaard,r_philosophers,8,"[1813, 1, 5]","Copenhagen, Denmark"
9,Friedrich_Nietzsche,r_philosophers,9,"[1844, 1, 15, 10:00]","Roecken, Germany"


In [183]:
geolocator = Nominatim(user_agent="specify_your_app_name_here")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
best_df_ever['location'] = best_df_ever['birthplace'].apply(geocode)
best_df_ever['point'] = best_df_ever['location'].apply(lambda loc: tuple(loc.point) if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('France',), **{}).
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/anaconda3/lib/python3.7/http/client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1016, in _send_output
    self.send(msg)
  File "/anaconda3/lib/python3.7/http/client.py", line 956, in send
    self.connect()
  File "/anaconda3/lib/python3.7/http/client.py", line 1384, in connect
    super().connect()
  File "/anaconda3/lib/python3.7/http/client.py", line 9

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Knnu, Plva County, Estonia',), **{}).
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/anaconda3/lib/python3.7/http/client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1016, in _send_output
    self.send(msg)
  File "/anaconda3/lib/python3.7/http/client.py", line 956, in send
    self.connect()
  File "/anaconda3/lib/python3.7/http/client.py", line 1384, in connect
    super().connect()
  File "/anaconda3/lib/python3.7/htt

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Prague, Czech Republic',), **{}).
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/anaconda3/lib/python3.7/http/client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1016, in _send_output
    self.send(msg)
  File "/anaconda3/lib/python3.7/http/client.py", line 956, in send
    self.connect()
  File "/anaconda3/lib/python3.7/http/client.py", line 1384, in connect
    super().connect()
  File "/anaconda3/lib/python3.7/http/cl

In [184]:
best_df_ever.tail(25)

,name,occupation,count,birthtime,birthplace,location,point
188,Nicholas_Roerich,r_philosophers,188,"[1874, 1, 9]","Saint Petersburg, Russia","(Санкт-Петербург, Северо-Западный федеральный ...","(59.9606739, 30.1586551, 0.0)"
189,Michele_Federico_Sciacca,r_philosophers,189,"[1908, 1, 12, 20:30]","Giarre, Italy","(Giarre, Catania, Sicilia, 95014, Italia, (37....","(37.726857, 15.187688, 0.0)"
190,Junípero_Serra,r_philosophers,190,"[1713, 1, 24, 01:00]","Majorca, Spain","(Mallorca, Sencelles, Pla de Mallorca, Illes B...","(39.6134018, 2.8804305326400756, 0.0)"
191,Paul_Tillich,r_philosophers,191,"[1886, 1, 20, 00:30]","Starosiedle, Poland","(Starosiedle, gmina Gubin, powiat krośnieński,...","(51.8581096, 14.820994, 0.0)"
192,Johannes_Vaihinger,r_philosophers,192,"[1852, 1, 25, 16:30]","hringen, Germany",None,None
193,Theobold_Ziegler,r_philosophers,193,"[1846, 1, 9, 17:45]","Gppingen, Germany",None,None
194,Ryszard_Antoni_Legutko,r_philosophers,194,"[1949, 1, 24]","Krakw, Poland",None,None
195,Miloslav_Ransdorf,r_philosophers,195,"[1953, 1, 15]","Rakovnk, Czech Republic",None,None
196,Karel_Josef_Beneš,r_philosophers,196,"[1896, 1, 21]","Prague, Czech Republic","(Praha, okres Hlavní město Praha, Hlavní město...","(50.0874654, 14.4212535, 0.0)"
197,Karel_Bodlák,r_philosophers,197,"[1903, 1, 3]","Sobslav, Czech Republic",None,None


This seems to work. We will perform this process on every occupation dataframe that we have.

But before we run a for loop on all our dataframes, let's continue with this specific data frame.

Let's feed our dataframe into FlatLib, get some results, analyze the results, and add these results to our dataframe as a set of columns.

I'm gonna use an np.array.

Ehhh... let's just stick to Pandas, it'll be just as direct.

In [185]:
#OLD CODE

'''
date = Datetime('1997/07/02', '16:07', '-07:00')
#date.jd
print(date.jd)

pos = GeoPos('37n7644', '121w9540')
print(pos.lat,pos.lon)

chart = Chart(date, pos)
'''
print('hi')

hi


In [186]:
jackson_df = pd.DataFrame([['Jackson_Stenger','r_philosopher',0,[1997, 7, 2],'San Ramon, California, United States']], columns = ['name', 'occupation', 'count', 'birthtime', 'birthplace'])


In [187]:
jackson_df.head()

,name,occupation,count,birthtime,birthplace
0,Jackson_Stenger,r_philosopher,0,"[1997, 7, 2]","San Ramon, California, United States"


In [188]:
geolocator1 = Nominatim(user_agent="specify_your_app_name_here")
geocode1 = RateLimiter(geolocator1.geocode, min_delay_seconds=1)
jackson_df['location'] = jackson_df['birthplace'].apply(geocode1)
jackson_df['point'] = jackson_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [189]:
jackson_df

,name,occupation,count,birthtime,birthplace,location,point
0,Jackson_Stenger,r_philosopher,0,"[1997, 7, 2]","San Ramon, California, United States","(San Ramon, Contra Costa County, California, U...","(37.7799273, -121.9780153, 0.0)"


In [190]:
df_var = best_df_ever
#df_var = jackson_df

def point_formatter(row):
    p = df_var.iloc[row].point
    if p == None:
        return None
    p_ns = p[0]
    p_ew = p[1]
    if p_ns > 0:
        p_ns_mod = str(round(p_ns, 4)).replace('.', 'n')
    else:
        p_ns_mod = str(round(-p_ns, 4)).replace('.', 's')
    if p_ew > 0:
        p_ew_mod = str(round(p_ew, 4)).replace('.', 'e')
    else:
        p_ew_mod = str(round(-p_ew, 4)).replace('.', 'w')
    return (p_ns_mod,p_ew_mod)

In [191]:
point_formatter(0)

('48n1371', '11e5754')

In [192]:
best_df_everer = best_df_ever
best_df_everer['point'] = best_df_ever['count'].apply(point_formatter)

In [193]:
best_df_everer.head(8)

,name,occupation,count,birthtime,birthplace,location,point
0,Michael_Haneke,r_philosophers,0,"[1942, 1, 23]","Munich, Germany","(München, Bayern, Deutschland, (48.1371079, 11...","(48n1371, 11e5754)"
1,Noam_Chomsky,r_philosophers,1,"[1928, 1, 7, 07:00]","Philadelphia, Pennsylvania, United States","(Philadelphia, Philadelphia County, Pennsylvan...","(39n9527, 75w1635)"
2,Albert_Camus,r_philosophers,2,"[1913, 1, 7, 02:00]","Mondovi, Algeria","(Drean, Daïra Drean, El Tarf الطارف, الجزائر, ...","(36n6851, 7e7471)"
3,Mahatma_Gandhi,r_philosophers,3,"[1869, 1, 2, 07:08]","Porbandar, India","(Porbandar, Porbandar Taluka, Porbandar Distri...","(21n6409, 69e611)"
4,Jean-Jacques_Rousseau,r_philosophers,4,"[1712, 1, 28, 10:00]","Geneva, Switzerland","(Genève, Switzerland, (46.2017559, 6.1466014))","(46n2018, 6e1466)"
5,Denis_Diderot,r_philosophers,5,"[1713, 1, 5, 17:00]","Langres, France","(Langres, Haute-Marne, Grand Est, France métro...","(47n8656, 5e3314)"
6,Immanuel_Kant,r_philosophers,6,"[1724, 1, 22, 05:00]","Knigsberg, Germany",None,None
7,Johann_Gottfried_Herder,r_philosophers,7,"[1744, 1, 25, 23:30]","Mohrungen, Poland","(Morąg, gmina Morąg, powiat ostródzki, wojewód...","(53n912, 19e928)"


In [222]:
#Method which returns the time offset for a particular timezone.

def get_offset(lat, lng):
    """
    returns a location's time zone offset from UTC in minutes.
    """
    tf = TimezoneFinder()

    today = datetime.now()
    tz_target = timezone(tf.certain_timezone_at(lng=lng, lat=lat))
    today_target = tz_target.localize(today)
    today_utc = utc.localize(today)
    val = (today_utc - today_target).total_seconds()/60
    hr = int(val // 60)
    hr = str(hr).zfill(2)
    mn = int(val - 60 * int(hr))
    mn = str(mn).zfill(2)
    ans = hr + ':' + mn
    return ans


#bergamo = {'lat': 45.69, 'lng': 9.67}
#minute_offset = get_offset(**bergamo)
#print(minute_offset)

In [223]:
print(get_offset(45.69,9.67))

02:00


In [333]:
var_df = best_df_everer
#var_df = jackson_df
##get_offset(lat, lng)
def date_formatter(row):
    old_date = var_df.iloc[row].birthtime
    
    year = str(old_date[0]).zfill(4)
    month = str(old_date[1]).zfill(2)
    day = str(old_date[2]).zfill(2)
    new_date = str(year) + '/' + str(month) + '/' + str(day)
    
    location = var_df.iloc[row].point
    lat = location[0]
    lng = location[1]
    if 'n' in lat:
        lat = lat.replace('n', '.')
    if 's' in lat:
        lat = '-' + (lat.replace('s', '.'))
    if 'e' in lng:
        lng = lng.replace('e', '.')
    if 'w' in lng:
        lng = '-' + (lng.replace('w', '.'))
    offset = get_offset(float(lat), float(lng))
    
    if len(old_date) == 4:
        time = str(old_date[3])
        ans = Datetime(new_date, time, offset)
    else:
        ans = Datetime(new_date, offset)
    return ans

In [334]:
date_formatter(0)

In [338]:
var_df = best_df_everer
#var_df = jackson_df

def chart_creator(row):
    if df_var.iloc[row].point == None or df_var.iloc[row].location == None or df_var.iloc[row].birthplace == None or df_var.iloc[row].birthtime == None:
        return None
    d = date_formatter(row)
    #^Remember to use the same df here.
    l = df_var.iloc[row].point
    pos = GeoPos(l[0], l[1])
    try:
        chart = Chart(d, pos)
        return chart
    except:
        None

In [340]:
#chart_creator(1)
#chart_creator(4).get(const.MOON)
chart_creator(0)

It works!!!

Now I'm gonna add a column filled with birthcharts (as objects).

In [341]:
best_df_everer['chart'] = best_df_ever['count'].apply(chart_creator)

In [342]:
best_df_everer.head()

,name,occupation,count,birthtime,birthplace,location,point,chart
0,Michael_Haneke,r_philosophers,0,"[1942, 1, 23]","Munich, Germany","(München, Bayern, Deutschland, (48.1371079, 11...","(48n1371, 11e5754)",None
1,Noam_Chomsky,r_philosophers,1,"[1928, 1, 7, 07:00]","Philadelphia, Pennsylvania, United States","(Philadelphia, Philadelphia County, Pennsylvan...","(39n9527, 75w1635)",<flatlib.chart.Chart object at 0x617e648d0>
2,Albert_Camus,r_philosophers,2,"[1913, 1, 7, 02:00]","Mondovi, Algeria","(Drean, Daïra Drean, El Tarf الطارف, الجزائر, ...","(36n6851, 7e7471)",<flatlib.chart.Chart object at 0x617e7d6a0>
3,Mahatma_Gandhi,r_philosophers,3,"[1869, 1, 2, 07:08]","Porbandar, India","(Porbandar, Porbandar Taluka, Porbandar Distri...","(21n6409, 69e611)",<flatlib.chart.Chart object at 0x617f0a518>
4,Jean-Jacques_Rousseau,r_philosophers,4,"[1712, 1, 28, 10:00]","Geneva, Switzerland","(Genève, Switzerland, (46.2017559, 6.1466014))","(46n2018, 6e1466)",None


I need to figure out how many 'None' charts I have.

In [345]:
#best_df_everer.info(verbose=True)
best_df_everer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 8 columns):
name          213 non-null object
occupation    213 non-null object
count         213 non-null int64
birthtime     205 non-null object
birthplace    205 non-null object
location      183 non-null object
point         183 non-null object
chart         136 non-null object
dtypes: int64(1), object(7)
memory usage: 13.4+ KB


I'm thinking at this point I should get rid of any rows that have no chart.

In [346]:
best_df_everest = best_df_everer

In [347]:
# df[df['column name'].map(lambda x: str(x)!=".")]
best_df_everest = best_df_everest[best_df_everest['chart'].map(lambda x: x!=None)]


In [349]:
best_df_everest.head(20)

,name,occupation,count,birthtime,birthplace,location,point,chart
1,Noam_Chomsky,r_philosophers,1,"[1928, 1, 7, 07:00]","Philadelphia, Pennsylvania, United States","(Philadelphia, Philadelphia County, Pennsylvan...","(39n9527, 75w1635)",<flatlib.chart.Chart object at 0x617e648d0>
2,Albert_Camus,r_philosophers,2,"[1913, 1, 7, 02:00]","Mondovi, Algeria","(Drean, Daïra Drean, El Tarf الطارف, الجزائر, ...","(36n6851, 7e7471)",<flatlib.chart.Chart object at 0x617e7d6a0>
3,Mahatma_Gandhi,r_philosophers,3,"[1869, 1, 2, 07:08]","Porbandar, India","(Porbandar, Porbandar Taluka, Porbandar Distri...","(21n6409, 69e611)",<flatlib.chart.Chart object at 0x617f0a518>
5,Denis_Diderot,r_philosophers,5,"[1713, 1, 5, 17:00]","Langres, France","(Langres, Haute-Marne, Grand Est, France métro...","(47n8656, 5e3314)",<flatlib.chart.Chart object at 0x617f0ae80>
7,Johann_Gottfried_Herder,r_philosophers,7,"[1744, 1, 25, 23:30]","Mohrungen, Poland","(Morąg, gmina Morąg, powiat ostródzki, wojewód...","(53n912, 19e928)",<flatlib.chart.Chart object at 0x617f0ab70>
8,Soeren_Kierkegaard,r_philosophers,8,"[1813, 1, 5]","Copenhagen, Denmark","(København, Københavns Kommune, Region Hovedst...","(55n6867, 12e5701)",<flatlib.chart.Chart object at 0x617f2d1d0>
9,Friedrich_Nietzsche,r_philosophers,9,"[1844, 1, 15, 10:00]","Roecken, Germany","(Röcken, Lützen, Burgenlandkreis, Sachsen-Anha...","(51n242, 12e1143)",<flatlib.chart.Chart object at 0x617f2d2e8>
10,Bertrand_Russell,r_philosophers,10,"[1872, 1, 18, 17:45]","Trelleck, Wales, United Kingdom","(Trelleck Grange, Devauden, Monmouthshire, Wal...","(51n7111, 2w7356)",<flatlib.chart.Chart object at 0x617f3eef0>
11,Philip_K._Dick,r_philosophers,11,"[1928, 1, 16, 12:15]","Chicago, Illinois, United States","(Chicago, Cook County, Illinois, United States...","(41n8756, 87w6244)",<flatlib.chart.Chart object at 0x617e50320>
13,Karl_Marx,r_philosophers,13,"[1818, 1, 5, 02:00]","Trier, Germany","(Trier, Rheinland-Pfalz, 54290, Deutschland, (...","(49n7596, 6e6442)",<flatlib.chart.Chart object at 0x617f3ed30>


Sweet, it works!!

I'm just gonna export this as a csv.

In [350]:
best_df_everest.to_csv('r_philosophers_charted.csv', index=False)